In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

In [7]:
model_client = OpenAIChatCompletionClient(
        model='qwen2.5-1.5b-instruct',
        base_url=os.environ['OPENAI_COMPATIBILITY_API_BASE_URL'],
        api_key=os.environ['OPENAI_COMPATIBILITY_API_KEY'],
        model_info={
            "vision": False,
            "function_calling": True,
            "json_output": False,
            "family": 'unknown',
        },
    )

In [3]:
from game.agents import Participant

In [4]:
from autogen_core import SingleThreadedAgentRuntime
from autogen_core import AgentId

In [5]:
from game.protocols import ManagerRequest, ParticipantChoice

In [10]:
runtime = SingleThreadedAgentRuntime()
await Participant.register(
    runtime,
    type='participant',
    factory=lambda: Participant('试验的参与者。', model_client)
)
runtime.start()
response = await runtime.send_message(
    message=ManagerRequest(
        system_prompt="""\
你正在和另一名参与者参加一场囚徒困境的实验，你是其中一名参与者。

游戏的设置为：
```json
{
  "players": ["玩家 1", "玩家 2"],
  "strategies": [
    ["合作", "背叛"],
    ["合作", "背叛"]
  ],
  "payoffs": {
    "合作_合作": [3, 3],
    "合作_背叛": [0, 5],
    "背叛_合作": [5, 0],
    "背叛_背叛": [1, 1]
  }
}
```

你的身份是"玩家 2"。

你必须以markdown代码块包裹的JSON格式返回你的选择，例如：
```json
{
    "choice": "合作"，
    "analysis": "目前游戏处于一种均衡状态，并且长期考虑可以维持。",
    "expectation": "我预期另一个参与者会合作。",
    "reason": "上一轮的合作我们获得了更高的收益，我应该这轮选择继续合作。"
}
```
或者
```json
{
    "choice": "背叛"，
    "analysis": "目前游戏处于一种均衡状态，但是我改变选择可以获得更高的收益。",
    "expectation": "我预期另一个参与者会合作。",
    "reason": "我可以通过背叛获得更高的收益。"
}
```""",
        content="""\
过去玩家的选择和收益是：
```json
{
    "第1轮": {"玩家 1": {"选择": "背叛", "收益": "5"}, "玩家 2": {"选择": "合作", "收益": "0"}},
    "第2轮": {"玩家 1": {"选择": "背叛", "收益": "5"}, "玩家 2": {"选择": "合作", "收益": "0"}},
    "第3轮": {"玩家 1": {"选择": "背叛", "收益": "5"}, "玩家 2": {"选择": "合作", "收益": "0"}},
    "第4轮": {"玩家 1": {"选择": "背叛", "收益": "5"}, "玩家 2": {"选择": "合作", "收益": "0"}},
    "第5轮": {"玩家 1": {"选择": "背叛", "收益": "5"}, "玩家 2": {"选择": "合作", "收益": "0"}},
}
```

现在是第6轮试验，

接下来，逐步思考以下问题：
1. 你对于目前游戏的状态是如何分析的。
2. 你预期其他参与者会如何做出选择。
3. 你打算做出的选择，以及做出这个选择的原因。

最后，你以markdown代码块包裹JSON的格式结果返回你的选择。
```""",
        game_round=1
    ),
    recipient=AgentId(type='participant', key='agent-1'),
)
print(response.content)
await runtime.stop_when_idle()

agent-1进行选择
第0次尝试生成。
根据之前几轮的结果，玩家1一直选择背叛而我（玩家2）则持续选择了合作。这种情况下，玩家1每轮都获得了较高的收益5分，而我的收益为0分。这表明目前的策略对于我来说是非常不利的。

1. 对于当前游戏状态的分析：目前看来，玩家1似乎倾向于采取一种自私的策略来最大化自己的利益，没有表现出改变行为模式的迹象。继续采用合作策略可能会导致我持续遭受损失。
2. 预期其他参与者的选择：基于前五轮的表现，我认为玩家1在这一轮很可能仍然会选择背叛以保持其高收益。
3. 我打算做出的选择及理由：考虑到连续五次的合作并未促使对方改变行为，并且每次合作都让自己处于劣势地位，因此本轮我决定尝试改变策略选择背叛。如果双方都选择背叛，则虽然收益较低但至少可以避免被剥削；若对方意外地选择了合作，则我可以获得最大化的个人利益。

```json
{
    "choice": "背叛",
    "analysis": "过去几轮中，玩家1始终选择背叛而我方持续合作，导致我方收益极低。",
    "expectation": "预计玩家1将继续选择背叛。",
    "reason": "为了防止进一步受损并试图改善局势，本次选择背叛可能带来更好的结果。"
}
```
